# 🌊 FlowGrad v0.4 — Complete Demo

**Training Diagnostics, Feature Engineering, Compression, Saliency, Quantization, Knowledge Distillation & LoRA/PEFT.**

Run all cells in order.

In [ ]:
# Install FlowGrad (Colab / fresh env)
# !pip install git+https://github.com/hw01931/FlowGrad.git
# !pip install torch xgboost lightgbm scikit-learn

In [ ]:
import flowgrad
flowgrad.info()

---
## Section 1: PyTorch FlowTracker — DL Training Diagnostics

In [ ]:
import torch
import torch.nn as nn
from flowgrad import FlowTracker

# Simple model
model = nn.Sequential(
    nn.Linear(20, 64),
    nn.BatchNorm1d(64),
    nn.ReLU(),
    nn.Dropout(0.3),
    nn.Linear(64, 32),
    nn.ReLU(),
    nn.Linear(32, 1),
)

optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=50)

tracker = FlowTracker(
    model,
    optimizer=optimizer,
    scheduler=scheduler,
    run_name="exp_01_baseline"
)

# Simulate training
X = torch.randn(128, 20)
y = torch.randn(128, 1)

for epoch in range(50):
    pred = model(X)
    loss = nn.MSELoss()(pred, y)
    loss.backward()
    optimizer.step()
    scheduler.step()
    optimizer.zero_grad()
    tracker.step(loss=loss.item())

tracker.report()

In [ ]:
# Visual dashboard
tracker.plot.full_report()

---
## Section 2: AI Agent Mode — Structured XML Output

In [ ]:
# Export training context as XML for AI assistants
xml = tracker.export_for_agent(save=False)
print(xml)

---
## Section 3: Dynamic Saliency — Intelligent Pruning Priority

In [ ]:
from flowgrad import SaliencyAnalyzer

sa = SaliencyAnalyzer(tracker)
sa.report()

# Individual analyses
print("\n--- Velocity Saliency ---")
for name, score in sa.velocity_saliency().items():
    print(f"  {name}: {score:.3f}")

print("\n--- Gradient Momentum ---")
for name, slope in sa.gradient_momentum().items():
    trend = "declining ↓" if slope < 0 else "growing ↑"
    print(f"  {name}: {slope:.6f} ({trend})")

---
## Section 4: Quantization Advisor — Mixed-Precision Guidance

In [ ]:
from flowgrad import QuantizationAdvisor

qa = QuantizationAdvisor(tracker)
qa.report()

print("\n--- Mixed-Precision Plan ---")
for layer, bits in qa.recommend_mixed_precision().items():
    print(f"  {layer}: {bits}-bit")

print(f"\nEstimated savings: {qa.estimated_size_reduction()}")

---
## Section 5: Model Compression — Auto Search

In [ ]:
from flowgrad import CompressionTracker

# Create a fresh model for compression
comp_model = nn.Sequential(
    nn.Linear(20, 64),
    nn.ReLU(),
    nn.Linear(64, 32),
    nn.ReLU(),
    nn.Linear(32, 1),
)

# Train it
opt = torch.optim.Adam(comp_model.parameters(), lr=0.01)
for _ in range(30):
    pred = comp_model(X)
    loss = nn.MSELoss()(pred, y)
    loss.backward()
    opt.step()
    opt.zero_grad()

# Compression eval function
def eval_fn(m):
    with torch.no_grad():
        pred = m(X)
        return -nn.MSELoss()(pred, y).item()  # higher = better

comp = CompressionTracker(comp_model, eval_fn=eval_fn)
result = comp.auto_compress(performance_floor=0.95)
comp.report()

---
## Section 6: Knowledge Distillation — Teacher-Student Diagnostics

In [ ]:
from flowgrad import DistillationTracker

# Teacher (large) and Student (small)
teacher = nn.Sequential(
    nn.Linear(20, 128), nn.ReLU(),
    nn.Linear(128, 64), nn.ReLU(),
    nn.Linear(64, 1),
)
student = nn.Sequential(
    nn.Linear(20, 32), nn.ReLU(),
    nn.Linear(32, 16), nn.ReLU(),
    nn.Linear(16, 1),
)

t_tracker = FlowTracker(teacher, run_name="teacher")
s_tracker = FlowTracker(student, run_name="student")

# Train teacher
t_opt = torch.optim.Adam(teacher.parameters(), lr=0.01)
for _ in range(30):
    pred = teacher(X)
    loss = nn.MSELoss()(pred, y)
    loss.backward()
    t_opt.step()
    t_opt.zero_grad()
    t_tracker.step(loss=loss.item())

# Train student with KD
s_opt = torch.optim.Adam(student.parameters(), lr=0.01)
for _ in range(30):
    with torch.no_grad():
        t_out = teacher(X)
    s_out = student(X)
    task_loss = nn.MSELoss()(s_out, y)
    kd_loss = nn.MSELoss()(s_out, t_out)
    loss = 0.5 * task_loss + 0.5 * kd_loss
    loss.backward()
    s_opt.step()
    s_opt.zero_grad()
    s_tracker.step(loss=loss.item())

dt = DistillationTracker(t_tracker, s_tracker)
dt.report()

In [ ]:
# Suggested KD loss weights (higher = student struggles more there)
weights = dt.suggest_distillation_weights()
print("Suggested KD loss weights per layer:")
for layer, w in weights.items():
    print(f"  {layer}: {w:.4f}")

---
## Section 7: LoRA / PEFT Diagnostics

In [ ]:
from flowgrad import PEFTTracker

# Simulate a model with LoRA-style adapters
class LoRAModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.base = nn.Linear(20, 64)
        self.lora_A = nn.Linear(20, 4, bias=False)  # Low-rank
        self.lora_B = nn.Linear(4, 64, bias=False)   # Low-rank
        self.head = nn.Linear(64, 1)

    def forward(self, x):
        h = self.base(x) + self.lora_B(self.lora_A(x))
        return self.head(torch.relu(h))

lora_model = LoRAModel()
lora_tracker = FlowTracker(lora_model, run_name="lora_exp")

# Freeze base, only train LoRA
for p in lora_model.base.parameters():
    p.requires_grad = False

lora_opt = torch.optim.Adam(filter(lambda p: p.requires_grad, lora_model.parameters()), lr=0.01)
for _ in range(30):
    pred = lora_model(X)
    loss = nn.MSELoss()(pred, y)
    loss.backward()
    lora_opt.step()
    lora_opt.zero_grad()
    lora_tracker.step(loss=loss.item())

pt = PEFTTracker(lora_tracker)
pt.report()

In [ ]:
# Rank recommendations
ranks = pt.recommend_ranks()
print("Recommended LoRA ranks:")
for layer, rank in sorted(ranks.items(), key=lambda x: -x[1]):
    print(f"  {layer}: rank={rank}")

---
## Section 8: XGBoost / Boosting Tracker

In [ ]:
from flowgrad import BoostingTracker
from sklearn.datasets import make_classification

X_cls, y_cls = make_classification(n_samples=500, n_features=10, random_state=42)

try:
    import xgboost as xgb
    
    bt = BoostingTracker()
    dtrain = xgb.DMatrix(X_cls, label=y_cls, feature_names=[f'f{i}' for i in range(10)])
    dval = xgb.DMatrix(X_cls[:100], label=y_cls[:100], feature_names=[f'f{i}' for i in range(10)])
    
    model = xgb.train(
        {'max_depth': 4, 'eta': 0.1, 'objective': 'binary:logistic', 'eval_metric': 'logloss'},
        dtrain,
        num_boost_round=100,
        evals=[(dtrain, 'train'), (dval, 'valid')],
        callbacks=[bt.as_xgb_callback()],
        verbose_eval=False,
    )
    bt.report()
except ImportError:
    print("XGBoost not installed — skipping.")

---
## Section 9: Feature Engineering + VIF

In [ ]:
from flowgrad import FeatureAnalyzer
from sklearn.ensemble import RandomForestClassifier
import numpy as np

rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_cls, y_cls)

feature_names = [f'feature_{i}' for i in range(10)]
analyzer = FeatureAnalyzer(rf, X_cls, y_cls, feature_names=feature_names)

# Interactions
interactions = analyzer.interactions(top_k=5)
print("Top interactions:")
for item in interactions:
    print(f"  {item['feat_a']} × {item['feat_b']}: synergy={item['synergy_score']:.4f}")

# Suggestions with VIF filter
suggestions = analyzer.suggest_features(top_k=5, collinearity_check=True)
print("\nFeature suggestions (VIF-filtered):")
for s in suggestions:
    print(f"  {s['expression']}: lift={s['lift']:.4f}")

---
## Section 10: All Agent XML Outputs

In [ ]:
# Each module can export its own XML section for AI agents
print("=" * 50)
print("SALIENCY:")
print(sa.to_agent_xml())
print()
print("QUANTIZATION:")
print(qa.to_agent_xml())
print()
print("DISTILLATION:")
print(dt.to_agent_xml())
print()
print("PEFT:")
print(pt.to_agent_xml())

In [ ]:
print("\n🎉 FlowGrad v0.5 — All features working!")
flowgrad.info()